In [127]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import glob
from nltk.stem.porter import PorterStemmer
from itertools import chain #untuk flaten array 2d

In [128]:
def readFile(directory):
    listFile = glob.glob(directory+"/*.txt")
    return listFile

## normalisasi

In [129]:
def normalisasi(body):

    #hapus tag
    hapusTag = r'<DATE>|</DATE>|<TITLE>|</TITLE>|<BODY>|</BODY>'
    body = re.sub(hapusTag, ' ', body)

    #hapus \n
    hapusEnter = r'\n'
    body = re.sub(hapusEnter, ' ', body)

    # hapus htmlEntities 
    htmlEntities = r"&lt;"
    body = re.sub(htmlEntities, '', body)

    kutip2 = r'[\"\(\)\<\>\+\?]'
    body = re.sub(kutip2, '', body)

    # hapus titik dan koma pada akhir string
    titikDiAkhirString = r"([a-zA-Z]+)([^\w\s])"
    temp = re.findall(titikDiAkhirString, body)
    x = True
    while x :
        print(temp)
        temp = re.findall(titikDiAkhirString, body)
        if len(temp) == 0 :
            x = False
        for i in temp:
            temp2 = i[0] + i[1]
            body = re.sub(temp2, i[0], body)

    # hapus titik dan koma pada akhir angka    
    titikSetelahAngka = r'(\d+)(^\w)\s'    
    temp = re.findall(titikSetelahAngka,body)
    while x :
        print(temp)
        temp = re.findall(titikSetelahAngka, body)
        if len(temp) == 0:
            x = False
        for i in temp:
            temp2 = i[0] + i[1]
            body = re.sub(temp2, i[0], body)

    kutips = r"'s"
    body = re.sub(kutips, '', body)
    
    spasiLebihDariSatu = r'\s+'
    body = re.sub(spasiLebihDariSatu, ' ', body)
    return body

In [130]:
def readText(file):
    words = []
    f = open(file, 'r') #open file
    text = f.read()    
    f.close()

    return text

In [131]:
def removeEnter(listOfWords): #['<BODY>\n']
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].replace('\n','')
        
    return listOfWords

In [132]:
def removeNull(listOfWords): #['','','eat','food','','']
    listOfWords = list(filter(None, listOfWords))
    
    return listOfWords

In [133]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

<h3>Load Stop Word from NLTK</h3>

In [134]:
stop_words = stopwords.words('english')

In [135]:
def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

<h3>Load Lemmatizer from NLTK</h3>

In [136]:
lemmatizer = WordNetLemmatizer()

In [137]:
def lemmatization(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = lemmatizer.lemmatize(listOfWords[i])
        
    return listOfWords

<h3>Load Stemming from NLTK</h3>

In [138]:
def stemming(listOfWords):
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in listOfWords]
    
    return stemmed

# Token

In [139]:
def make_token(txt):
    list_hasil = txt.split(" ")            
    return list_hasil

# Term

In [140]:
def make_term(file_name_token,file_name_term):
    list_term = []
    f = open(file_name_token, 'r')
    text = f.read()
    f.close()
    
    list_token = text.split("\n")
    for i in list_token:
        if i not in list_term:
            list_term.append(i)
            
    f1 = open(file_name_term,'a+')
    text = concate_term_newline(list_term)
    print(text)    
    f1.write(text)
    f1.close()


In [141]:
def concate_term_newline(list_in):
    hasil = ''
    for i in list_in:
        hasil = hasil + "\n " + i
        
    return hasil

# tulis data

In [142]:
def writeFile(list_in,file_name):
    list_flatten = list(chain.from_iterable(list_in))
        
    str_of_list = concate_term_newline(list_flatten)
    
    saveFileToken = open(file_name, 'a+' )
    saveFileToken.write(str_of_list)
    saveFileToken.close()
#     print(list_hasil)
    

<h2>Add Doc Number</h2>

In [143]:
def addDocNumber(_listIn, _docNum):
#     _docNum[14:18]
    for i in range(len(_listIn)):
        _listIn[i] = _listIn[i] + '_' + _docNum[14:18]
        
    return _listIn

<h1>Main</h1>

In [144]:
listOfFiles = readFile('DataRouter')
listOfWordsOfFile = []
counter = 1
blok = 10
list_token_sebelum = []
list_token_sesudah = []
list_term_sebelum = []
list_term_sesudah = []
blok_file_name = "term_blok"
blok_name = []

for i in listOfFiles: #iterasi tiap file
    txt = readText(i)
    list_token_sebelum.append(make_token(txt))
    txt = normalisasi(txt)
    listOfWords = make_token(txt)
    listOfWords = removeNull(listOfWords)
    listOfWords = caseFolding(listOfWords)
    listOfWords = removeStopWord(listOfWords)
    listOfWords = lemmatization(listOfWords)
#     listOfWords = stemming(listOfWords)
    
    # Diganti KEVIN
    listOfWords = addDocNumber(listOfWords,i)
    
    list_token_sesudah.append(listOfWords)
    counter = counter + 1
#     print(counter)

    if (counter%blok) == 0:        
        writeFile(list_token_sebelum,"token_sebelum_pemprosesan.txt")
        writeFile(list_token_sesudah,"token_sesudah_pemprosesan.txt")
        
        #melakukan inisialisasi list dari, agar tidak memakan terlalu banyak memory
        listofWords = []
        list_token_sebelum = []
        list_token_sesudah = []
        list_term_sebelum = []
        list_term_sesudah = []
        
make_term("token_sebelum_pemprosesan.txt","term_sebelum_pemprosesan.txt")
make_term("token_sesudah_pemprosesan.txt","term_sesudah_pemprosesan.txt")
    

 accounted_0491
  19_0491
  from_0491
  6.5_0491
  1985._0491
  the_0491
  product_0491
  slated_0491
  include_0491
  automobile_0491
  machinery_0491
  part_0491
  catalytic_0491
  converter_0491
  fuel_0491
  injection_0491
  system_0491
  electronic_0491
  emission_0491
  testing_0491
  said_0491
  26-feb1987_0492
  17:55:36.88_0492
  banksexpres_0492
  grave_0492
  concern_0492
  brazil_0492
  debt_0492
  move_0492
  14-bank_0492
  advisory_0492
  committee_0492
  expressed_0492
  chief_0492
  negotiator_0492
  antonio_0492
  padua_0492
  de_0492
  sixas_0492
  country_0492
  suspension_0492
  interest_0492
  payment_0492
  according_0492
  telex_0492
  chairman_0492
  citibank_0492
  creditor_0492
  bank_0492
  worldwide_0492
  banker_0492
  saidthe_0492
  diplomatic_0492
  phrase_0492
  belied_0492
  deep_0492
  anger_0492
  frustration_0492
  unilateral_0492
  last_0492
  friday_0492
  subsequent_0492
  freeze_0492
  some_0492
  15_0492
  billion_0492
  dlrs_0492
  shortterm_04